In [121]:
from sites import sites
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import datetime

конвертация месяц-номер

In [2]:
month_lib = {'января':'01','февраля':'02','марта':'03','апреля':'04','мая':'05','июня':'06','июля':'07','августа':'08','сентября':'09','октября':'10','ноября':'11','декабря':'12'}

### Выполняемсть сайтов  
<ul>
<li><s>УРГУПС (URUST) - 'https://www.usurt.ru/main-news'</s></li>
<li><s>УИ ГА (UVAUGA) - 'https://uvauga.ru'</s></li>
<li>УГА (SPBGUGA) - 'https://spbguga.ru/category/news/'</li>
<li><s>СГУПС (STU) - 'http://www.stu.ru/news/'</s></li>
<li>(SSUWT) -</li>
<li>РГУПС (RGUPS) - 'https://www.rgups.ru/news/'</li>  
<li>ПГУПС (PGUPS) - 'https://pgups.ru/news/'</li>
<li>ОМГУПС (OMGUPS) - 'https://www.omgups.ru/news/'</li> 
<li>ИГРУПС (IRGUPS) - 'https://www.irgups.ru/news'</li>
<li>ДВГУПС (DVGUPS) - 'https://dvgups.ru/press-main-page'</li>  
<li>ГУМРФ (GUMRF) - 'https://gumrf.ru/news/'</li>
<li>САМГУПС (SAMGUPS) - 'https://www.samgups.ru/?PAGEN_1='</li>
<li>МГУ им. адм. Ф. Ф. Ушакова (AUMSU) - 'http://pk.aumsu.ru/novosti'</li>
<li>МГУ им. адм. Г.И. Невельского (MSUN) - 'https://www.msun.ru/ru/news/arch'</li>

## <a id='1'>URUST</a>


In [135]:
page = requests.get('https://www.usurt.ru/main-news')
print('_-'*70, '\n urust __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')


soup = bs(page.text, "html.parser") #варим суп по странице всех новостей
all_hrefs = soup.findAll('a', class_="news__button-more") #вытаскиваем все элементы с ссылками на отдельные новости

all_dates = soup.find_all('div', attrs={'style':'font-weight: 700; margin-bottom: 10px;'}) #даты
posts_usurt = {}
key = 'dt'
for date in all_dates:
    date = str(date.text)
    date = date[6:8]+'/'+date[3:5]+'/'+date[0:2]
    posts_usurt.setdefault(key, [])
    posts_usurt[key].append(date)

all_img_hrefs = soup.find_all('div',  class_='news-row__preview-img') #ссылки на картинки 
key = 'img_href'
for img_href in all_img_hrefs:
    img_href = img_href.get('style')
    img_href = img_href[23:-3]
    
    posts_usurt.setdefault(key, [])
    posts_usurt[key].append(img_href)

for link in all_hrefs: 
    href = link.get('href') #вытаскиваем ссылки из элементов поочередно
    news_link = f'https://www.usurt.ru{href}' #достраиваем ссылку до полной
    page_news = requests.get(news_link) #запрос на индивидуальную новость
    
    print(news_link, '__connection status__ -', page_news.status_code, '\n', '_'*50, '\n') #проверка связи


    news_soup = bs(page_news.text, "html.parser") #варим новый суп из странички с новостью

    key = 'name'
    name = news_soup.find_all('h1')[1] #вытаскиваем элемент заголовка
    # print("Заголовок\n", name.text, '\n', '-'*20)
    posts_usurt.setdefault(key, [])
    posts_usurt[key].append(name.text)

    key = 'text'
    box = news_soup.find_all('span', attrs={'style':'font-size:18px;'}) #вытягиваем все элементы с текстом
    text = '' #переменная для конечного результата
    for elem in box: #перебираем элементы поочередно и вытаскивает текст
        text+=elem.text
    # print("Основной текст\n", text, '\n', '-'*30, '\n\n')
    posts_usurt.setdefault(key, [])
    posts_usurt[key].append(text)   


_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 urust __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 


https://www.usurt.ru/main-news/news-20482 __connection status__ - 200 
 __________________________________________________ 

https://www.usurt.ru/main-news/news-20481 __connection status__ - 200 
 __________________________________________________ 

https://www.usurt.ru/main-news/news-20480 __connection status__ - 200 
 __________________________________________________ 

https://www.usurt.ru/main-news/news-20479 __connection status__ - 200 
 __________________________________________________ 

https://www.usurt.ru/main-news/news-20478 __connection status__ - 200 
 __________________________________________________ 

https://www.usurt.ru/main-news/news-20477 __connecti

In [136]:
posts_usurt = pd.DataFrame.from_dict(posts_usurt)
posts_usurt

,dt,img_href,name,text
0,23/10/13,http://www.usurt.ru/uploads/main/0dd/6528ce428...,Болеем за наших!,Вчера стартовали Всероссийские соревнования ср...
1,23/10/12,http://www.usurt.ru/uploads/main/0dd/6527debbe...,Выберем место для фандомата вместе,Дорогие друзья! В ближайшее время в Уральском ...
2,23/10/12,http://www.usurt.ru/uploads/main/0dd/6527cb703...,В Екатеринбурге начала работу патриотическая с...,"Советник директора КЖТ УрГУПС, руководитель ре..."
3,23/10/12,http://www.usurt.ru/uploads/main/0dd/6527c6ec0...,Кубок первокурсника,18 октября в 14:00 состоится Кубок первокурсни...
4,23/10/11,http://www.usurt.ru/uploads/main/0dd/652668f7a...,УрГУПС примет участие в профориентационной выс...,Дорогие абитуриенты!Уже в эти выходные в Екате...
5,23/10/11,http://www.usurt.ru/uploads/main/0dd/65263ea45...,Студенты УрГУПС приняли участие в «Осеннем кро...,Студенты Уральского государственного университ...
6,23/10/11,http://www.usurt.ru/uploads/main/0dd/65262cc27...,Приглашаем на встречу с членом Совета директор...,13 октября в 12:00 в аудитории Б4-75 состоится...
7,23/10/10,http://www.usurt.ru/uploads/main/0dd/652531096...,Профориентационная встреча,Сегодня в общеобразовательной школе № 45 город...
8,23/10/10,http://www.usurt.ru/uploads/main/0dc/6524cee46...,В УрГУПС приехали эксперты,9 октября в Уральском государственном универси...
9,23/10/09,http://www.usurt.ru/uploads/main/0dc/6523789f6...,Центр защиты прав молодёжи,В 2023 году запущен Центр защиты прав молодежи...


In [137]:
posts_usurt.to_csv('posts/posts_usurt.csv')

## STU

In [142]:
posts_stu = {}

for i in range(59):    
    url = f'http://www.stu.ru/news/index.php?page=527&podrazd=&p={i}'
    
    page = requests.get(url)
    print('_-'*70, '\n urust __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')


    soup = bs(page.text, "html.parser") #варим суп по странице всех новостей

    
    hrefs = soup.find_all('a', class_=('newsLink'))
    all_img_hrefs = soup.find_all('img', class_=('no_zoom_plz'))

    key = 'img_href'
    for img_href in all_img_hrefs:
        img_href = img_href.get('src')
        img_href = f'http://www.stu.ru{img_href}'
        posts_stu.setdefault(key, [])
        posts_stu[key].append(img_href)


    for href in hrefs:
        link = href.get('href')
        link = f'http://www.stu.ru/news/{link}'

        link_page = requests.get(link)
        link_soup = bs(link_page.text, "html.parser")


        div = link_soup.find_all('div', {'id':'page_text'})


        key = 'dt'
        for date in div:
            date = date.find('p').text
            date = date[0:10]
            date = date[8:10]+'/'+date[3:5]+'/'+date[0:2]
            posts_stu.setdefault(key, [])
            posts_stu[key].append(date)


        key = 'name'
        for name in div:
            name = name.find('p').text
            name = name[11:-1]
            posts_stu.setdefault(key, [])
            posts_stu[key].append(name)

            

        key = 'text'
        text_div = link_soup.find_all('p', attrs={'style':'margin-top: 1em;'})
        for textus in text_div:
            textus = textus.text
            posts_stu.setdefault(key, [])
            posts_stu[key].append(textus)

    print('страница ',i,'\n')


_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 urust __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 


страница  0 

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 urust __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 


страница  1 

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 urust __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 


стр

In [143]:
posts_stu = pd.DataFrame.from_dict(posts_stu)
posts_stu

,img_href,dt,name,text
0,http://www.stu.ru/images/news/preview/48906.jpg,23/10/13,Студенты СГУПСа борются за Кубок ректора по ГТ,С 11 октября на спортивных сооружениях и в бас...
1,http://www.stu.ru/images/news/preview/48905.jpg,23/10/13,Студенты университета начали традиционную убор...,Дважды в год – весной и осенью – по приказу ре...
2,http://www.stu.ru/images/news/preview/48904.jpg,23/10/12,Победа студентов в конкурсе грантов,В 2023 году Фонд содействия инновациям провел ...
3,http://www.stu.ru/images/news/preview/48903.jpg,23/10/12,СГУПС принял участие в образовательном форуме ...,Форум прошел 10 октября в ДК «Прогресс» и на н...
4,http://www.stu.ru/images/news/preview/48902.jpg,23/10/11,«Механики» СГУПСа - чемпионы России по специал...,"В конце сентября, накануне юбилея факультета У..."
...,...,...,...,...
585,http://www.stu.ru/images/news/preview/48271.jpg,21/05/17,В СГУПСе прошли конференции в рамках «Дни наук...,Этой весной на базе СГУПСа кафедра «Таможенное...
586,http://www.stu.ru/images/news/preview/48270.jpg,21/05/14,Подведены итоги II (Регионального) этапа Всеро...,Подведены итоги II (Регионального) этапа Всеро...
587,http://www.stu.ru/images/news/preview/48269.jpg,21/05/14,СГУПС выиграл во Всероссийском конкурсе проект...,В Росмолодежи на заседании были утверждены поб...
588,http://www.stu.ru/images/news/preview/48268.jpg,21/05/12,"В СГУПСе прошли праздничные мероприятия, посвя...",8 мая в 21:00 СГУПС принял участие в общенаци...


In [144]:
posts_stu.to_csv('posts/posts_stu.csv')

## UVAUGA

In [153]:
page = requests.get('https://uvauga.ru')
print('_-'*70, '\n uvauga __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

soup = bs(page.text, "html.parser")
posts_uvauga = {}


#Сбор всех ссылок на новости (Основная страница + нижние "другие новости")

hrefs = soup.find_all("p", class_='readmore')
count = 1
all_hrefs = []

key = 'name'
for href in hrefs: #ссылки на индивидуальные новости
    name = href.text
    href = href.find("a").get("href")
    href = f'https://uvauga.ru/{href}'
    # print(f'{count}', href)
    # count += 1
    all_hrefs.append(href)
    posts_uvauga.setdefault(key, [])
    posts_uvauga[key].append(name)
    
    

hrefs_more = soup.find_all('div', class_='items-more')

for href_more in hrefs_more:
    href_more = href_more.find_all("a")

    for href_rez in href_more:
        name = href_rez.text
        href_rez = href_rez.get('href')
        href_rez = f'https://uvauga.ru{href_rez}'
        # print(f'{count}', href_rez)
        # count+=1
        all_hrefs.append(href_rez)
        posts_uvauga.setdefault(key, [])
        posts_uvauga[key].append(name)


_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 uvauga __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [154]:
#Залезим в новости
for href in all_hrefs:
    page = requests.get(href)
    print('_-'*70, f'\n {href} __connection status__ -', page.status_code, '\n', '_-'*30, '\n\n')

    if page.status_code != 200:
        print(f'Ошибка подключения к странице {href}')
        break

    soup = bs(page.text, "html.parser")

    key = 'text'
    texts = soup.find_all('div', class_='item-page')
    all_imgs = texts
    for text in texts:
        text = text.text
        print(text,'_-'*20,'\n')
        posts_uvauga.setdefault(key, [])
        posts_uvauga[key].append(text)
    key = 'imgs'
    for imgs in all_imgs:
        imgs = imgs.find_all('img')
        img_rez = []
        if imgs != None:
            for img in imgs:
                img = img.get('src')
                img = f'https://uvauga.ru/{img}'
                img_rez.append(img)
            print(img_rez)

        posts_uvauga.setdefault(key, [])
        posts_uvauga[key].append(img_rez)



_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 https://uvauga.ru//2137-lgotysirotam __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 



ВНИМАНИЕ! Курсанты, обучающиеся за счет бюджетных ассигнований федерального бюджета по очной форме обучения, имеющие статус детей-сирот и детей, оставшихся без попечения родителей, имеют право на получение  дополнительных гарантий из федерального бюджета по социальной поддержке
(либо, по желанию обучающегося, на получение денежной компенсации)
согласно ФЗ от 21.12.1996 года № 159-ФЗ «О дополнительных гарантиях по социальной поддержке детей-сирот и детей, оставшихся без попечения родителей»:
- обеспечение бесплатным питанием;
- обеспечение бесплатным комплектом одежды, обуви и мягким инвентарём;
- выплату пособия на приобретение учебной литературы и письменных принадлежностей;
- возмещение расходов на проезд в городс

In [158]:
print('_-'*50,'\n')
print(len(all_hrefs))
print(len(posts_uvauga['name']))
print(len(posts_uvauga['text']))
print(len(posts_uvauga['imgs']))
print(posts_uvauga['imgs'])

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 

32
32
32
32
0                                                    []
1     [https://uvauga.ru//images/MainNews/VAZNO.jpg,...
2                                                    []
3     [https://uvauga.ru//images/du.jpg, https://uva...
4     [https://uvauga.ru//images/OAB/logotip_oab.jpg...
5     [https://uvauga.ru//images/MainNews/navigatorn...
6     [https://uvauga.ru//images/MainNews/donor-09-2...
7     [https://uvauga.ru//images/MainNews/rkfkis.jpg...
8       [https://uvauga.ru//images/MainNews/rucode.jpg]
9     [https://uvauga.ru//images/MainNews/kurs-mol-b...
10    [https://uvauga.ru//images/MainNews/vhod-test-...
11                 [https://uvauga.ru//images/TTds.jpg]
12    [https://uvauga.ru//images/MainNews/otkryty-be...
13    [https://uvauga.ru//images/MainNews/3Spept/3-0...
14    [https://uvauga.ru//images/MainNews/obluniver2...
15    [https://uvauga.ru//images/MainNews/VF-

In [159]:
posts_uvauga = pd.DataFrame.from_dict(posts_uvauga)
posts_uvauga

,name,text,imgs
0,\n\n\t\t\t\t\tО льготах...\n,"\nВНИМАНИЕ! Курсанты, обучающиеся за счет бюдж...",[]
1,\n\n\t\t\t\t\tО переводе или зачислении гражда...,\n\nПо вопросам приема заявлений о переводе и...,"[https://uvauga.ru//images/MainNews/VAZNO.jpg,..."
2,\n\n\t\t\t\t\tВременное проживание в РФ в целя...,"\nИнформация для иностранных граждан, планирую...",[]
3,\n\n\t\t\t\t\tУчить. Вдохновлять. Развивать.\n,\n\n06 октября 2023 года в читальном зале науч...,"[https://uvauga.ru//images/du.jpg, https://uva..."
4,\n\n\t\t\t\t\tVII молодежная научно-практическ...,\n\nКафедра обеспечения авиационной безопаснос...,[https://uvauga.ru//images/OAB/logotip_oab.jpg...
5,\n\n\t\t\t\t\tВидеоматериалы проекта «Наука и ...,\n\n \nВЫБЕРИ СВОЮ НАУЧНУЮ СПЕЦИАЛЬНОСТЬНАУЧНУ...,[https://uvauga.ru//images/MainNews/navigatorn...
6,\n\n\t\t\t\t\tОб участии во Всероссийском мара...,\n\nОб участии во Всероссийском марафоне донор...,[https://uvauga.ru//images/MainNews/donor-09-2...
7,"\n\n\t\t\t\t\tII Всероссийская конференция ""Ф...","\n\n20-21 июня в г. Москва, прошла II Всеросси...",[https://uvauga.ru//images/MainNews/rkfkis.jpg...
8,\n\n\t\t\t\t\tМероприятия Всероссийского фести...,\n\nВсероссийский фестиваль RuCode по искусств...,[https://uvauga.ru//images/MainNews/rucode.jpg]
9,\n\n\t\t\t\t\tВсероссийский историко-просветит...,\n\nВсероссийский историко-просветительский фо...,[https://uvauga.ru//images/MainNews/kurs-mol-b...


In [115]:
posts_uvauga.to_csv('posts/posts_uvauga.csv')

## SPBGUGA

In [ ]:
url = 'https://spbguga.ru/category/news/'

page = requests.get(url)
print('_-'*70, '\n spbguga __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 spbguga __connection status__ - 500 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [ ]:
print(page.status_code)

500


## MIIT

In [15]:
url = 'https://www.miit.ru/news?from=1&to=50&category_id=1383,1384,1127,1363' #посты с 1 по 50 страницу

page = requests.get(url)
print('_-'*70, '\n miit __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')


_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 miit __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [16]:
soup = bs(page.text,"html.parser")

In [17]:
posts = soup.find_all("div", class_=('container'))
print(posts)

[<div class="container">
<div class="clearfix">
<a class="header__logo" href="/">
<img src="/content/logo_flagstripe_ministryeagle_ministry_eagle_rut_2.svg?id_wm=900277" style="height: 50px;"/>
</a>
<ul class="pull-right list-unstyled list-inline">
<li>
<div class="dropdown">
<a aria-expanded="false" aria-haspopup="true" class="dropdown-toggle" data-toggle="dropdown" href="#" id="langSwitcher" role="button">
<svg style="height: 26px; width: 31px;">
<use xlink:href="/content/collection2_07.svg?id_wm=848520#icon-ru-1"></use>
</svg>
                            RU
    <svg class="rotate_90 hover_60 icon_arrow">
<use xlink:href="/content/svg-sprite-miit.svg?id_wm=909430#icon-arrow"></use>
</svg>
</a>
<div aria-labelledby="langSwitcher" class="dropdown-menu">
<a class="dropdown-item active" href="/">
<svg style="height: 26px; width: 31px;">
<use xlink:href="/content/collection2_07.svg?id_wm=848520#icon-ru-1"></use>
</svg>
Русский
</a><a class="dropdown-item" href="/en">
<svg style="height: 2

## SSUWT

In [19]:
url = "https://www.ssuwt.ru/?limit=12&limitstart=0"

page = requests.get(url)
print('_-'*70, '\n ssuwt __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

KeyboardInterrupt: 

In [ ]:
soup = bs(page.text, "html.parser")
hmp = soup.find('div', class_='k2PaginationCounter')
hmp = int(hmp.text[27:-8])

In [ ]:
posts_ssuwt = {}
for i in range(0,(hmp-1)*12,12):

    url = f"https://www.ssuwt.ru/?limit=12&limitstart={i}"
    page = requests.get(url)
    print('_-'*70, f'\n ssuwt_{int((i/12)+1)} __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

    soup = bs(page.text, "html.parser")


    key = 'dt'
    all_dt = soup.find_all('span', class_=('catItemDateCreated'))
    for date in all_dt:
        date = date.text[13:-8].split(',')[1]

        day = date.split()[0]
        month = month_lib[date.split()[1]]
        year = date.split()[2]

        time = date.split()[3]

        date = day+'.'+month+'.'+year+' '+time

        posts_ssuwt.setdefault(key, [])
        posts_ssuwt[key].append(date)


    h3 = soup.find_all('h3', class_=('catItemTitle'))

    
    key = 'name'
    for post in h3:
        elem = post.find('a')

        name = elem.text[17:-12]
        posts_ssuwt.setdefault(key, [])
        posts_ssuwt[key].append(name)


    key = 'text'
    for post in h3:
        elem = post.find('a')

        href = elem.get('href')
        link = f'https://www.ssuwt.ru/{href}'

        link_page = requests.get(link)
        soup = bs(link_page.text, "html.parser")

        textus = soup.find_all('div', class_=('itemFullText'))
        for text in textus:
            text = text.text

            posts_ssuwt.setdefault(key, [])
            posts_ssuwt[key].append(text)

    

    print(posts_ssuwt)


In [ ]:
posts_ssuwt = pd.DataFrame.from_dict(posts_ssuwt)
posts_ssuwt

## RGUPS

In [20]:
url = 'http://www.rgups.ru/news/page1'

page = requests.get(url)
print('_-'*70, '\n rgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 rgups __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [21]:
soup = bs(page.text, 'html.parser')

## PGUPS

In [22]:
url = 'https://pgups.ru/news/?PAGEN_2=1'

page = requests.get(url)
print('_-'*70, '\n pgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

KeyboardInterrupt: 

In [ ]:
soup = bs(page.text, "html.parser")

## OMGUPS

In [23]:
soup = bs(page.text, "html.parser")

## IRGUPS

In [24]:
url = 'https://www.irgups.ru/news?page=1'

page = requests.get(url)
print('_-'*70, '\n irgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 irgups __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [25]:
soup = bs(page.text, "html.parser")

## DVGUPS

In [26]:
url = 'https://dvgups.ru/press-main-page?start=0'

page = requests.get(url)
print('_-'*70, '\n dvgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 dvgups __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [27]:
soup = bs(page.text, "html.parser")

## GUMRF

In [28]:
url = 'https://gumrf.ru/news2023/2'

page = requests.get(url)
print('_-'*70, '\n gumrf __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 gumrf __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [29]:
soup = bs(page.text, "html.parser")

## SAMGUPS

In [30]:
url = 'https://www.samgups.ru/?PAGEN_1=1'

page = requests.get(url)
print('_-'*70, '\n samgups __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 samgups __connection status__ - 403 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [31]:
soup = bs(page.text, "html.parser")

## AUMSU

In [33]:
url = 'http://pk.aumsu.ru/novosti'

page = requests.get(url)
print('_-'*70, '\n aumsu __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 aumsu __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [34]:
soup = bs(page.text, "html.parser")

## MSUN

In [36]:
url = 'https://www.msun.ru/ru/news/arch'

page = requests.get(url)
print('_-'*70, '\n msun __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 msun __connection status__ - 200 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [37]:
soup = bs(page.text, "html.parser")

## SPBGUGA

In [38]:
url = 'https://spbguga.ru/category/news/'

page = requests.get(url)
print('_-'*70, '\n spbguga __connection status__ -', page.status_code, '\n', '_-'*70, '\n\n')

_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 
 spbguga __connection status__ - 500 
 _-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_- 




In [ ]:
print(page.status_code)

500
